In [24]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regressions 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from statistics import stdev
from sklearn.metrics import mean_squared_error
import scipy.stats
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split


players=['S Mandhana', 'Shafali Verma', 'D Hemalatha', 'H Kaur', 'Richa Ghosh', 'JI Rodrigues', 'Deepti Sharma', 'P Vastrakar', 'A Reddy', 'RP Yadav', 'A Sobhana', 'P Punia', 'Renuka Singh Thakur', 'S Ishaque', 'S M Shakil', 'Shreyanka Patil', 'U Chetry', 'L Wolvaardt', 'Tazmin Brits', 'Anneke Bosch', 'Sune Luus', 'M Kapp', 'N de Klerk', 'N Shangase', 'M Ridder', 'M Klaas', 'N Mlaba', 'A Khaka', 'A Dercksen', 'S Jafta', 'E Marx', 'T Sekhukhune', 'D Tucker']
final_dati = pd.DataFrame()  # Final dataframe to store all the player data

driver = webdriver.Chrome()

for i in players:
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=ODI&groupby=match&playerStatsFilters=on&start_date=2022-01-01&end_date=2023-01-01&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)  # Let the page load
    
    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="ODI-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tab
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ODI-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="ODI-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_dati =  pd.concat([final_dati,merged_stats])
final_dati = final_dati[final_dati['Match']!=0]

final_dati['Bowl Avg'] = np.where(final_dati['Bowl Avg']=='-',0,final_dati['Bowl Avg'])
final_dati['Bowl SR'] = np.where(final_dati['Bowl SR']=='-',0,final_dati['Bowl SR'])
final_dati= final_dati[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_dati = final_dati.replace('-',0)

final_dati


,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,S Mandhana,2022_W_015,13,15,1,86.7,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,71,84,0.0,0.0,0.0
1,S Mandhana,2022_W_016,71,84,1,84.5,1,0,9,0,...,0.0,0.0,0.0,0.0,0.0,52,75,0.0,0.0,0.0
2,S Mandhana,2022_W_020,52,75,1,69.3,1,0,3,1,...,0.0,0.0,0.0,0.0,0.0,6,21,0.0,0.0,0.0
3,S Mandhana,2022_W_024,6,21,1,28.6,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,123,119,0.0,0.0,0.0
4,S Mandhana,2022_W_026,123,119,1,103.4,0,1,13,2,...,0.0,0.0,0.0,0.0,0.0,35,59,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,E Marx,2022_W_003,1,2,0,50.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,NaN
0,T Sekhukhune,2022_W_002,14,16,1,87.5,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,1,2,0.0,0.0,0.0
1,T Sekhukhune,2022_W_003,1,2,0,50.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,NaN
0,D Tucker,2022_W_052,0,0,0,0,0,0,0,0,...,0,0,3,0,43.8,14,8,3.0,6,1


In [25]:
dfff = pd.read_csv('INDvsSA.csv')

# Merge the CSV data with the existing DataFrame based on the "Name" column in CSV and "Player" column in the existing DataFrame
totale = pd.merge(final_dati, dfff, left_on='Player', right_on='Name', how='left')

# Drop the redundant "Name" column
totale.drop(columns=['Name'], inplace=True)
totale =totale.rename(columns={'Selected_By': 'Selection'})

# Display the merged DataFrame
totale.head()
totale=totale[['Player','Match','Runs Scored','Balls Played','Out','Bat SR','50','100','4s Scored','6s Scored','Bat Dot%','Overs Bowled','Runs Given','Wickets Taken','Econ','Bowl Avg','Bowl SR','5W','4s Given','6s Given','Bowl Dot%','next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Position','Credits','Selection','Points']]
totale

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Position,Credits,Selection,Points
0,S Mandhana,2022_W_015,13,15,1,86.7,0,0,2,0,...,0.0,71,84,0.0,0.0,0.0,BAT,8.5,72%,0
1,S Mandhana,2022_W_016,71,84,1,84.5,1,0,9,0,...,0.0,52,75,0.0,0.0,0.0,BAT,8.5,72%,0
2,S Mandhana,2022_W_020,52,75,1,69.3,1,0,3,1,...,0.0,6,21,0.0,0.0,0.0,BAT,8.5,72%,0
3,S Mandhana,2022_W_024,6,21,1,28.6,0,0,0,0,...,0.0,123,119,0.0,0.0,0.0,BAT,8.5,72%,0
4,S Mandhana,2022_W_026,123,119,1,103.4,0,1,13,2,...,0.0,35,59,0.0,0.0,0.0,BAT,8.5,72%,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,E Marx,2022_W_003,1,2,0,50.0,0,0,0,0,...,0.0,None,None,NaN,NaN,NaN,AR,8.0,94%,0
299,T Sekhukhune,2022_W_002,14,16,1,87.5,0,0,2,0,...,0.0,1,2,0.0,0.0,0.0,BOWL,8.0,94%,0
300,T Sekhukhune,2022_W_003,1,2,0,50.0,0,0,0,0,...,0.0,None,None,NaN,NaN,NaN,BOWL,8.0,94%,0
301,D Tucker,2022_W_052,0,0,0,0,0,0,0,0,...,43.8,14,8,3.0,6,1,BAT,8.0,94%,0


In [18]:
import pandas as pd
from flask import Flask, send_file

# Assuming final_data is your DataFrame

# Convert final_data to CSV
totale.to_csv('final_data.csv', index=False)

# Initialize Flask app
app = Flask(__name__)

@app.route('/download')
def download_file():
    # Send the CSV file to the user for download
    return send_file('final_data.csv', as_attachment=True)

if __name__ == '__main__':
    # Run the app
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2024 08:58:40] "GET /download HTTP/1.1" 200 -


In [26]:
dfdd=totale
dfdd = pd.get_dummies(dfdd, columns=['Position'], drop_first=True, prefix=['Position'])
dfdd = dfdd[['Player','Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%','next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts', 'Credits', 'Selection', 'Points']]
dfdd['Selection'] = dfdd['Selection'].str.replace('%', '')
dfdd[dfdd.columns[6:]] = dfdd[dfdd.columns[6:]].astype(float)
df_model = dfdd[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored','Balls Played','4s Scored', '6s Scored','Out', 'Bat SR','Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Selection', 'Credits','Points']]
# Add new columns
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
# Convert 'Runs Scored' column to numeric type
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')

# Add new columns
df_model.insert(loc=8, column='30 runs', value=(df_model['Runs Scored'] > 30).astype(int))
df_model.insert(loc=9, column='50 runs', value=(df_model['Runs Scored'] > 50).astype(int))
df_model.insert(loc=10, column='100 runs', value=(df_model['Runs Scored'] > 100).astype(int))
df_model.insert(loc=11, column='Duck', value=(df_model['Runs Scored'] == 0).astype(int))
# Insert new columns into df_model based on the values of 'Wickets Taken' column
df_model.insert(loc=18, column='3 wickets', value=np.where(df_model['Wickets Taken'] >= 3, 1, 0))
df_model.insert(loc=19, column='4 wickets', value=np.where(df_model['Wickets Taken'] >= 4, 1, 0))
df_model.insert(loc=20, column='5 wickets', value=np.where(df_model['Wickets Taken'] >= 5, 1, 0))

# Calculate points based on the provided rules
df_model['Points'] = (
    # announced
    df_model['Runs Scored'] +            # run
    df_model['4s Scored'] +              # boundary_bonus
    2 * df_model['6s Scored'] +          # six_bonus
    4 * df_model['30 runs'] +            # 30_run_bonus
    8 * (df_model['50 runs'] == 1) +     # half_century_bonus
    16 * (df_model['100 runs'] == 1) +   # century_bonus
    -2 * df_model['Duck'] +              # dismissal_for_duck
    25 * df_model['Wickets Taken'] +     # wicket
    8 * (df_model['Wickets Taken'] >= 3) +  # 3_wicket_haul
    8 * (df_model['Wickets Taken'] >= 4) +  # 4_wicket_haul
    16 * (df_model['Wickets Taken'] >= 5) + # 5_wicket_haul
        # maiden
                # catch
     # runout(DirectHit/Stumping)
      # runout(Catcher/Thrower)
    ((df_model['Overs Bowled'] >= 2) & (df_model['Econ'] < 5)) * 6 +     # economy_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] > 170)) * 6 + # strike_rate_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 150.01) & (df_model['Bat SR'] <= 170)) * 4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 130) & (df_model['Bat SR'] <= 150)) * 2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 60) & (df_model['Bat SR'] <= 70)) * -2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 50) & (df_model['Bat SR'] <= 59.99)) * -4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] < 50)) * -6
)

# Display the DataFrame with new 'Points' column
df_model.head()



df_model

C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1447843031.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1447843031.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1447843031.py:23: SettingWithCopyWarning: 
A value is trying to be set 

,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,4 wickets,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Points
0,S Mandhana,True,False,False,13,15.0,2.0,0.0,0,0,...,0,0,71.0,84.0,0.0,0.0,0.0,72.0,8.5,15.0
1,S Mandhana,True,False,False,71,84.0,9.0,0.0,1,1,...,0,0,52.0,75.0,0.0,0.0,0.0,72.0,8.5,92.0
2,S Mandhana,True,False,False,52,75.0,3.0,1.0,1,1,...,0,0,6.0,21.0,0.0,0.0,0.0,72.0,8.5,67.0
3,S Mandhana,True,False,False,6,21.0,0.0,0.0,0,0,...,0,0,123.0,119.0,0.0,0.0,0.0,72.0,8.5,0.0
4,S Mandhana,True,False,False,123,119.0,13.0,2.0,1,1,...,0,0,35.0,59.0,0.0,0.0,0.0,72.0,8.5,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,E Marx,False,False,False,1,2.0,0.0,0.0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,94.0,8.0,1.0
299,T Sekhukhune,False,True,False,14,16.0,2.0,0.0,0,0,...,0,0,1.0,2.0,0.0,0.0,0.0,94.0,8.0,16.0
300,T Sekhukhune,False,True,False,1,2.0,0.0,0.0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,94.0,8.0,1.0
301,D Tucker,True,False,False,0,0.0,0.0,0.0,0,0,...,0,0,14.0,8.0,3.0,6.0,1.0,94.0,8.0,-2.0


In [27]:
df_model.fillna(0, inplace=True)

C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\579083050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.fillna(0, inplace=True)


In [28]:

# Linear Regression
# Fitting the model and checking accuracy

X =df_model[df_model.columns [1:-1]]
y = df_model[df_model.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=9999)
points_model = LinearRegression().fit(X_train,y_train)
print('Training set accuracy:', points_model.score (X_train,y_train)) 
print('Test set accuracy:',points_model.score (X_test,y_test))


Training set accuracy: 0.9979760124836651
Test set accuracy: 0.9972728920361468


In [22]:
df_model.columns [1:]

Index(['Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored',
       'Balls Played', '4s Scored', '6s Scored', '30 runs', '50 runs',
       '100 runs', 'Duck', 'Out', 'Bat SR', 'Overs Bowled', 'Runs Given',
       'Wickets Taken', 'Econ', '3 wickets', '4 wickets', '5 wickets',
       'next_runs', 'next_balls', 'next_overs', 'next_runs_given', 'next_wkts',
       'Selection', 'Credits', 'Points'],
      dtype='object')

In [29]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regressions 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from statistics import stdev
from sklearn.metrics import mean_squared_error
import scipy.stats
import math

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
final_dat = pd.DataFrame()
players=['S Mandhana', 'Shafali Verma', 'D Hemalatha', 'H Kaur', 'Richa Ghosh', 'JI Rodrigues', 'Deepti Sharma', 'P Vastrakar', 'A Reddy', 'RP Yadav', 'A Sobhana', 'P Punia', 'Renuka Singh Thakur', 'S Ishaque', 'S M Shakil', 'Shreyanka Patil', 'U Chetry']
driver=webdriver.Chrome()
# Looping through all the players
#
for i in players[0:]:
    # Accessing the web page for the current player's stats
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=ODI&groupby=match&playerStatsFilters=on&start_date=2023-01-02&end_date=2024-06-22&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)

    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="ODI-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tab
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ODI-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="ODI-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_dat =  pd.concat([final_dat,merged_stats])
final_dat = final_dat[final_dat['Match']!=0]

final_dat['Bowl Avg'] = np.where(final_dat['Bowl Avg']=='-',0,final_dat['Bowl Avg'])
final_dat['Bowl SR'] = np.where(final_dat['Bowl SR']=='-',0,final_dat['Bowl SR'])
final_dat= final_dat[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_dat = final_dat.replace('-',0)
final_dat.fillna(0, inplace=True)
final_dat


,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,S Mandhana,2023_W_010,11,12,1,91.7,0,0,2,0,...,0,0,0,0,0,36,58,0,0,0
1,S Mandhana,2023_W_011,36,58,1,62.1,0,0,4,0,...,0,0,0,0,0,59,85,0,0,0
2,S Mandhana,2023_W_012,59,85,1,69.4,1,0,5,0,...,0,0,0,0,0,34,38,0,0,0
3,S Mandhana,2023_W_041,34,38,1,89.5,0,0,3,1,...,0,0,0,0,0,29,29,0,0,0
4,S Mandhana,2024_W_001,29,29,1,100.0,0,0,5,0,...,0,0,0,0,0,117,128,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,S M Shakil,2024_W_001,0,1,1,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
0,Shreyanka Patil,2023_W_041,5,2,0,250.0,0,0,1,0,...,60.0,0,4,0,55.0,2,10,10.0,57,3
1,Shreyanka Patil,2024_W_001,2,10,1,20.0,0,0,0,0,...,20.0,0,5,2,55.0,0,0,0,0,0
0,U Chetry,2023_W_041,5,2,0,250.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,2,10,0.0,0.0,0.0


In [31]:

players=['S Mandhana', 'Shafali Verma', 'D Hemalatha', 'H Kaur', 'Richa Ghosh', 'JI Rodrigues', 'Deepti Sharma', 'P Vastrakar', 'A Reddy', 'RP Yadav', 'A Sobhana', 'Renuka Singh Thakur']
models = pd.DataFrame()

for i in players:
    player = final_dat[final_dat['Player'] == i] 
    player_new = player.dropna()
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[21:22]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs = Ridge(alpha=k) 
    next_runs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[22:23]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_balls = Ridge(alpha=k)
    next_balls.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[25:26]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_wkts = Ridge(alpha=k) 
    next_wkts.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[23:24]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_overs = Ridge(alpha=k)
    next_overs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[24:25]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs_given = Ridge(alpha=k) 
    next_runs_given.fit(X_train, y_train)
    
    latest = player.groupby('Player').tail(1)
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])
    
    models = pd.concat([models,latest])

models['next_runs_given'] = round(models['next_runs_given'], 0)
models['next_overs'] = round(models['next_overs'], 0)
models['next_runs'] = round(models['next_runs'], 0)
models['next_balls'] = round(models['next_balls'], 0)
models['next_wkts'] = round(models['next_wkts'], 0)

models




C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:241: LinAlgWarning: Ill-conditioned matrix (rcond=6.8935e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:241: LinAlgWarning: Ill-conditioned matrix (rcond=6.8935e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:243: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:243: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\anilk\AppData\Local\Programs\

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
6,S Mandhana,2024_W_030,136,120,1,113.3,0,1,18,2,...,12.0,0,1,1,58.3,46.0,42.0,0.0,0.0,0.0
3,Shafali Verma,2024_W_030,20,38,1,52.6,0,0,3,0,...,0,0,1,1,29.2,68.0,128.0,1.0,4.0,0.0
1,D Hemalatha,2024_W_030,24,41,1,58.5,0,0,1,2,...,0.0,0.0,0.0,0.0,0.0,24.0,41.0,0.0,0.0,0.0
7,H Kaur,2024_W_030,103,88,0,117.0,0,1,9,3,...,0,0,0,0,0,-2.0,-1.0,1.0,12.0,0.0
4,Richa Ghosh,2024_W_030,25,13,0,192.3,0,0,3,1,...,0.0,0.0,0.0,0.0,0.0,10.0,12.0,0.0,0.0,0.0
6,JI Rodrigues,2024_W_029,17,28,1,60.7,0,0,1,0,...,0,0,0,0,0,18.0,18.0,1.0,1.0,1.0
7,Deepti Sharma,2024_W_030,0,0,0,0,0,0,0,0,...,30.0,0,4,2,53.3,3.0,-0.0,5.0,12.0,0.0
5,P Vastrakar,2024_W_030,0,0,0,0,0,0,0,0,...,21.0,0,8,1,42.9,2.0,1.0,2.0,10.0,0.0
4,A Reddy,2024_W_029,31,42,0,73.8,0,0,3,0,...,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
1,RP Yadav,2024_W_030,0,0,0,0,0,0,0,0,...,0,0,5,0,43.3,0.0,0.0,10.0,52.0,0.0


In [60]:
   
final_data = pd.DataFrame()
players=['L Wolvaardt', 'Tazmin Brits', 'Anneke Bosch', 'Sune Luus', 'M Kapp', 'N de Klerk', 'N Shangase', 'M Ridder', 'M Klaas', 'N Mlaba', 'A Khaka', 'A Dercksen', 'S Jafta', 'E Marx', 'T Sekhukhune', 'D Tucker']
final_dati = pd.DataFrame()  # Final dataframe to store all the player data
driver=webdriver.Chrome()
# Looping through all the players
#
for i in players[0:]:
    # Accessing the web page for the current player's stats
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=ODI&groupby=match&playerStatsFilters=on&start_date=2023-01-02&end_date=2024-06-22&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)

    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="ODI-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tabA
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ODI-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="ODI-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_data =  pd.concat([final_data,merged_stats])
final_data = final_data[final_data['Match']!=0]

final_data['Bowl Avg'] = np.where(final_data['Bowl Avg']=='-',0,final_data['Bowl Avg'])
final_data['Bowl SR'] = np.where(final_data['Bowl SR']=='-',0,final_data['Bowl SR'])
final_data= final_data[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_data = final_data.replace('-',0)
final_data.fillna(0, inplace=True)
final_data




,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,L Wolvaardt,2023_W_022,17,15,1,113.3,0,0,3,0,...,0.0,0.0,0.0,0.0,0.0,13,23,0.0,0.0,0.0
1,L Wolvaardt,2023_W_023,13,23,1,56.5,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0,4,0.0,0.0,0.0
2,L Wolvaardt,2023_W_024,0,4,1,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,50,75,0.0,0.0,0.0
3,L Wolvaardt,2023_W_028,50,75,1,66.7,1,0,3,0,...,0.0,0.0,0.0,0.0,0.0,124,141,0.0,0.0,0.0
4,L Wolvaardt,2023_W_029,124,141,0,87.9,0,1,15,0,...,0.0,0.0,0.0,0.0,0.0,24,54,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,D Tucker,2023_W_024,25,28,1,89.3,0,0,4,0,...,0,0,2,0,54.2,14,21,10.0,43,1
3,D Tucker,2023_W_037,14,21,1,66.7,0,0,1,0,...,60.0,0,5,0,70.0,8,14,0,0,0
4,D Tucker,2024_W_014,8,14,0,57.1,0,0,0,0,...,0,0,0,0,0,2,4,6.0,26,0
5,D Tucker,2024_W_015,2,4,1,50.0,0,0,0,0,...,0,0,2,0,50.0,1,11,5.0,42,1


In [74]:
players=['L Wolvaardt', 'Tazmin Brits', 'Anneke Bosch', 'Sune Luus', 'M Kapp', 'N de Klerk', 'N Shangase', 'M Klaas', 'N Mlaba', 'A Khaka', 'S Jafta', 'E Marx', 'D Tucker']
models1=pd.DataFrame()

for i in players:
    player = final_data[final_data['Player'] == i] 
    player_new = player.dropna()
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[21:22]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs = Ridge(alpha=k) 
    next_runs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[22:23]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_balls = Ridge(alpha=k)
    next_balls.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[25:26]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_wkts = Ridge(alpha=k) 
    next_wkts.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[23:24]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_overs = Ridge(alpha=k)
    next_overs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[24:25]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs_given = Ridge(alpha=k) 
    next_runs_given.fit(X_train, y_train)
    
    latest = player.groupby('Player').tail(1)
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])
    
    models1 = pd.concat([models1,latest])

models1['next_runs_given'] = round(models1['next_runs_given'], 0)
models1['next_overs'] = round(models1['next_overs'], 0)
models1['next_runs'] = round(models1['next_runs'], 0)
models1['next_balls'] = round(models1['next_balls'], 0)
models1['next_wkts'] = round(models1['next_wkts'], 0)

models1


C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1947692504.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1947692504.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1947692504.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
16,L Wolvaardt,2024_W_030,135,135,0,100.0,0,1,12,3,...,0.0,0.0,0.0,0.0,0.0,124.0,107.0,0.0,0.0,0.0
14,Tazmin Brits,2024_W_030,5,11,1,45.5,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0
9,Anneke Bosch,2024_W_030,18,23,1,78.3,0,0,3,0,...,0,0,2,0,33.3,19.0,24.0,0.0,0.0,0.0
16,Sune Luus,2024_W_030,12,13,1,92.3,0,0,2,0,...,0,0,2,2,33.3,43.0,57.0,0.0,3.0,0.0
15,M Kapp,2024_W_030,114,94,1,121.3,0,1,11,3,...,0,0,0,0,0,-5.0,-7.0,2.0,4.0,1.0
15,N de Klerk,2024_W_030,28,22,1,127.3,0,0,2,1,...,0,0,5,1,50.0,13.0,12.0,1.0,11.0,-0.0
2,N Shangase,2024_W_030,0,1,1,0.0,0,0,0,0,...,0,0,5,2,30.6,0.0,-0.0,-0.0,-0.0,0.0
15,M Klaas,2024_W_030,0,0,0,0,0,0,0,0,...,60.0,0,7,2,63.3,2.0,5.0,3.0,24.0,-0.0
12,N Mlaba,2024_W_030,0,0,0,0,0,0,0,0,...,30.0,0,6,1,56.7,2.0,1.0,4.0,20.0,1.0
14,A Khaka,2024_W_030,0,0,0,0,0,0,0,0,...,0,0,7,0,68.3,2.0,3.0,6.0,25.0,-0.0


In [75]:
dfcf = pd.read_csv('INDvsSA.csv')
fdfs=pd.concat([models,models1])
# Merge the CSV data with the existing DataFrame based on the "Name" column in CSV and "Player" column in the existing DataFrame
totali = pd.merge(fdfs, dfcf, left_on='Player', right_on='Name', how='left')

# Drop the redundant "Name" column
totali.drop(columns=['Name'], inplace=True)
totali =totali.rename(columns={'Selected_By': 'Selection'})

# Display the merged DataFrame
totali.head()
totali=totali[['Player','Match','Runs Scored','Balls Played','Out','Bat SR','50','100','4s Scored','6s Scored','Bat Dot%','Overs Bowled','Runs Given','Wickets Taken','Econ','Bowl Avg','Bowl SR','5W','4s Given','6s Given','Bowl Dot%','next_runs', 'next_balls', 'next_overs','next_runs_given', 'next_wkts','Team','Position','Credits','Selection','Points']]
totali

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Team,Position,Credits,Selection,Points
0,S Mandhana,2024_W_030,136,120,1,113.3,0,1,18,2,...,46.0,42.0,0.0,0.0,0.0,IND,BAT,8.5,72%,0
1,Shafali Verma,2024_W_030,20,38,1,52.6,0,0,3,0,...,68.0,128.0,1.0,4.0,0.0,IND,BAT,8.0,68%,0
2,D Hemalatha,2024_W_030,24,41,1,58.5,0,0,1,2,...,24.0,41.0,0.0,0.0,0.0,IND,BAT,7.5,1%,0
3,H Kaur,2024_W_030,103,88,0,117.0,0,1,9,3,...,-2.0,-1.0,1.0,12.0,0.0,IND,BAT,7.5,85%,0
4,Richa Ghosh,2024_W_030,25,13,0,192.3,0,0,3,1,...,10.0,12.0,0.0,0.0,0.0,IND,WK,7.5,44%,0
5,JI Rodrigues,2024_W_029,17,28,1,60.7,0,0,1,0,...,18.0,18.0,1.0,1.0,1.0,IND,BAT,7.0,59%,0
6,Deepti Sharma,2024_W_030,0,0,0,0,0,0,0,0,...,3.0,-0.0,5.0,12.0,0.0,IND,AR,8.0,96%,0
7,P Vastrakar,2024_W_030,0,0,0,0,0,0,0,0,...,2.0,1.0,2.0,10.0,0.0,IND,AR,7.5,16%,0
8,A Reddy,2024_W_029,31,42,0,73.8,0,0,3,0,...,3.0,2.0,0.0,0.0,0.0,IND,BOWL,8.0,42%,0
9,RP Yadav,2024_W_030,0,0,0,0,0,0,0,0,...,0.0,0.0,10.0,52.0,0.0,IND,BOWL,6.0,4%,0


In [76]:
totali.columns

Index(['Player', 'Match', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50',
       '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled',
       'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W',
       '4s Given', '6s Given', 'Bowl Dot%', 'next_runs', 'next_balls',
       'next_overs', 'next_runs_given', 'next_wkts', 'Team', 'Position',
       'Credits', 'Selection', 'Points'],
      dtype='object')

In [77]:
dsfn=totali
dsfn = pd.get_dummies(dsfn, columns=['Position'], drop_first=True, prefix=['Position'])
dsfn = dsfn[['Player','Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Credits', 'Selection', 'Points']]
dsfn['Selection'] = dsfn['Selection'].str.replace('%', '')
dsfn[dsfn.columns[6:]] = dsfn[dsfn.columns[6:]].astype(float)
points_cal = dsfn[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored','Balls Played','4s Scored', '6s Scored','Out', 'Bat SR','Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Selection', 'Credits','Points']]
# Add new columns
points_cal['Runs Scored'] = pd.to_numeric(points_cal['Runs Scored'], errors='coerce')
# Convert 'Runs Scored' column to numeric type
points_cal['Runs Scored'] = pd.to_numeric(points_cal['Runs Scored'], errors='coerce')

# Add new columns
points_cal.insert(loc=8, column='30 runs', value=(points_cal['Runs Scored'] > 30).astype(int))
points_cal.insert(loc=9, column='50 runs', value=(points_cal['Runs Scored'] > 50).astype(int))
points_cal.insert(loc=10, column='100 runs', value=(df_model['Runs Scored'] > 100).astype(int))
points_cal.insert(loc=11, column='Duck', value=(points_cal['Runs Scored'] == 0).astype(int))
# Insert new columns into df_model based on the values of 'Wickets Taken' column
points_cal.insert(loc=18, column='3 wickets', value=np.where(points_cal['Wickets Taken'] >= 3, 1, 0))
points_cal.insert(loc=19, column='4 wickets', value=np.where(points_cal['Wickets Taken'] >= 4, 1, 0))
points_cal.insert(loc=20, column='5 wickets', value=np.where(points_cal['Wickets Taken'] >= 5, 1, 0))

# Calculate points based on the provided rules
points_cal['Points'] = (
    # announced
    points_cal['next_runs'] +            # run
    points_cal['4s Scored'] +              # boundary_bonus
    2 * points_cal['6s Scored'] +          # six_bonus
    4 * points_cal['30 runs'] +            # 30_run_bonus
    8 * (points_cal['50 runs'] == 1) +     # half_century_bonus
    16 * (points_cal['100 runs'] == 1) +   # century_bonus
    -2 * points_cal['Duck'] +              # dismissal_for_duck
    25 * points_cal['next_wkts'] +     # wicket
    8 * (points_cal['next_wkts'] >= 3) +  # 3_wicket_haul
    8 * (points_cal['next_wkts'] >= 4) +  # 4_wicket_haul
    16 * (points_cal['next_wkts'] >= 5) + # 5_wicket_haul
        # maiden
                # catch
     # runout(DirectHit/Stumping)
      # runout(Catcher/Thrower)
    ((points_cal['next_overs'] >= 2) & (points_cal['Econ'] < 5)) * 6 +     # economy_points
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] > 170)) * 6 + # strike_rate_points
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] >= 150.01) & (points_cal['Bat SR'] <= 170)) * 4 +
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] >= 130) & (points_cal['Bat SR'] <= 150)) * 2 +
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] >= 60) & (points_cal['Bat SR'] <= 70)) * -2 +
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] >= 50) & (points_cal['Bat SR'] <= 59.99)) * -4 +
    ((points_cal['next_balls'] >= 10) & (points_cal['Bat SR'] < 50)) * -6
)

# Display the DataFrame with new 'Points' column
points_cal.head()



points_cal

C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\2728706986.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_cal['Runs Scored'] = pd.to_numeric(points_cal['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\2728706986.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_cal['Runs Scored'] = pd.to_numeric(points_cal['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\2728706986.py:23: SettingWithCopyWarning: 
A value is trying to

,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,4 wickets,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Points
0,S Mandhana,True,False,False,136,120.0,18.0,2.0,1,1,...,0,0,46.0,42.0,0.0,0.0,0.0,72.0,8.5,80.0
1,Shafali Verma,True,False,False,20,38.0,3.0,0.0,0,0,...,0,0,68.0,128.0,1.0,4.0,0.0,68.0,8.0,67.0
2,D Hemalatha,True,False,False,24,41.0,1.0,2.0,0,0,...,0,0,24.0,41.0,0.0,0.0,0.0,1.0,7.5,25.0
3,H Kaur,True,False,False,103,88.0,9.0,3.0,1,1,...,0,0,-2.0,-1.0,1.0,12.0,0.0,85.0,7.5,25.0
4,Richa Ghosh,False,False,True,25,13.0,3.0,1.0,0,0,...,0,0,10.0,12.0,0.0,0.0,0.0,44.0,7.5,37.0
5,JI Rodrigues,True,False,False,17,28.0,1.0,0.0,0,0,...,0,0,18.0,18.0,1.0,1.0,1.0,59.0,7.0,42.0
6,Deepti Sharma,False,False,False,0,0.0,0.0,0.0,0,0,...,0,0,3.0,-0.0,5.0,12.0,0.0,96.0,8.0,1.0
7,P Vastrakar,False,False,False,0,0.0,0.0,0.0,0,0,...,0,0,2.0,1.0,2.0,10.0,0.0,16.0,7.5,0.0
8,A Reddy,False,True,False,31,42.0,3.0,0.0,1,0,...,0,0,3.0,2.0,0.0,0.0,0.0,42.0,8.0,10.0
9,RP Yadav,False,True,False,0,0.0,0.0,0.0,0,0,...,0,0,0.0,0.0,10.0,52.0,0.0,4.0,6.0,-2.0


In [78]:
X = df_model[df_model.columns[4:6]]
y = df_model[df_model.columns[6:7]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)
fours_model = LinearRegression().fit(X_train, y_train)
print('Training set accuracy:', fours_model.score(X_train, y_train))
print('Test set accuracy:', fours_model.score(X_test, y_test))

Training set accuracy: 0.796399589912271
Test set accuracy: 0.831014235267943


In [79]:
X = df_model[df_model.columns[4:6]]
y = df_model[df_model.columns[7:8]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=999)
sixes_model = LinearRegression().fit(X_train, y_train)
print("Training set accuracy:", sixes_model.score(X_train, y_train))
print('Test set accuracy:', sixes_model.score(X_test, y_test))

Training set accuracy: 0.35050922438085785
Test set accuracy: 0.28344903976783997


In [80]:

X = df_model[df_model.columns[1:12]]
y = df_model[df_model.columns[12:13]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9999)
out_model = LogisticRegression().fit(X_train, y_train)
print('Training set accuracy:', out_model.score(X_train, y_train))
print('Test set accuracy:', out_model.score(X_test, y_test))

Training set accuracy: 0.8237885462555066
Test set accuracy: 0.6710526315789473


C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [81]:
import pandas as pd

# Assuming you have totali_df and points_cal_df already loaded with your data

# Merge the two dataframes on the 'player' column to get corresponding points
merged_dfd = pd.merge(totali, points_cal[['Player', 'Points']], on='Player', how='left')

# Rename the points column from points_cal_df to avoid conflict
merged_dfd.drop(columns=['Points_x'], inplace=True, errors='ignore')
merged_dfd = merged_dfd.rename(columns={'Points_y': 'Points'})


# Now merged_df contains the 'points' column replaced with values from points_cal_df
# If you want, you can assign merged_df back to totali_df
merged_dfd


,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Team,Position,Credits,Selection,Points
0,S Mandhana,2024_W_030,136,120,1,113.3,0,1,18,2,...,46.0,42.0,0.0,0.0,0.0,IND,BAT,8.5,72%,80.0
1,Shafali Verma,2024_W_030,20,38,1,52.6,0,0,3,0,...,68.0,128.0,1.0,4.0,0.0,IND,BAT,8.0,68%,67.0
2,D Hemalatha,2024_W_030,24,41,1,58.5,0,0,1,2,...,24.0,41.0,0.0,0.0,0.0,IND,BAT,7.5,1%,25.0
3,H Kaur,2024_W_030,103,88,0,117.0,0,1,9,3,...,-2.0,-1.0,1.0,12.0,0.0,IND,BAT,7.5,85%,25.0
4,Richa Ghosh,2024_W_030,25,13,0,192.3,0,0,3,1,...,10.0,12.0,0.0,0.0,0.0,IND,WK,7.5,44%,37.0
5,JI Rodrigues,2024_W_029,17,28,1,60.7,0,0,1,0,...,18.0,18.0,1.0,1.0,1.0,IND,BAT,7.0,59%,42.0
6,Deepti Sharma,2024_W_030,0,0,0,0,0,0,0,0,...,3.0,-0.0,5.0,12.0,0.0,IND,AR,8.0,96%,1.0
7,P Vastrakar,2024_W_030,0,0,0,0,0,0,0,0,...,2.0,1.0,2.0,10.0,0.0,IND,AR,7.5,16%,0.0
8,A Reddy,2024_W_029,31,42,0,73.8,0,0,3,0,...,3.0,2.0,0.0,0.0,0.0,IND,BOWL,8.0,42%,10.0
9,RP Yadav,2024_W_030,0,0,0,0,0,0,0,0,...,0.0,0.0,10.0,52.0,0.0,IND,BOWL,6.0,4%,-2.0


In [82]:
predict=merged_dfd

In [83]:
predict

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Team,Position,Credits,Selection,Points
0,S Mandhana,2024_W_030,136,120,1,113.3,0,1,18,2,...,46.0,42.0,0.0,0.0,0.0,IND,BAT,8.5,72%,80.0
1,Shafali Verma,2024_W_030,20,38,1,52.6,0,0,3,0,...,68.0,128.0,1.0,4.0,0.0,IND,BAT,8.0,68%,67.0
2,D Hemalatha,2024_W_030,24,41,1,58.5,0,0,1,2,...,24.0,41.0,0.0,0.0,0.0,IND,BAT,7.5,1%,25.0
3,H Kaur,2024_W_030,103,88,0,117.0,0,1,9,3,...,-2.0,-1.0,1.0,12.0,0.0,IND,BAT,7.5,85%,25.0
4,Richa Ghosh,2024_W_030,25,13,0,192.3,0,0,3,1,...,10.0,12.0,0.0,0.0,0.0,IND,WK,7.5,44%,37.0
5,JI Rodrigues,2024_W_029,17,28,1,60.7,0,0,1,0,...,18.0,18.0,1.0,1.0,1.0,IND,BAT,7.0,59%,42.0
6,Deepti Sharma,2024_W_030,0,0,0,0,0,0,0,0,...,3.0,-0.0,5.0,12.0,0.0,IND,AR,8.0,96%,1.0
7,P Vastrakar,2024_W_030,0,0,0,0,0,0,0,0,...,2.0,1.0,2.0,10.0,0.0,IND,AR,7.5,16%,0.0
8,A Reddy,2024_W_029,31,42,0,73.8,0,0,3,0,...,3.0,2.0,0.0,0.0,0.0,IND,BOWL,8.0,42%,10.0
9,RP Yadav,2024_W_030,0,0,0,0,0,0,0,0,...,0.0,0.0,10.0,52.0,0.0,IND,BOWL,6.0,4%,-2.0


In [84]:
# Preparing current match's data in the relevant format and predicting the Dream11 points a player will score
test1 = predict

# Handle missing values in 'Bowl Avg' and 'Bowl SR' columns
test1['Bowl Avg'] = np.where(test1['Bowl Avg'] == '-', 0, test1['Bowl Avg'])
test1['Bowl SR'] = np.where(test1['Bowl SR'] == '', 0, test1['Bowl SR'])

# One-hot encode the 'Position' column
test1 = pd.get_dummies(test1, columns=['Position'], drop_first=True, prefix=['Position'])

# Select relevant columns for prediction
test1 = test1[['Player', 'Team', 'Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%', 'next_runs', 'next_balls', 'next_overs', 'next_runs_given', 'next_wkts', 'Credits', 'Selection', 'Points']]

# Clean 'Selection' column
test1['Selection'] = test1['Selection'].str.replace('%', '')

# Convert selected columns to float
test1[test1.columns[6:]] = test1[test1.columns[6:]].astype(float)

# Create a new DataFrame for prediction
test2 = test1[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK','Runs Scored', 'Balls Played','Overs Bowled', 'Runs Given', 'Wickets Taken','next_runs', 'next_balls','next_overs','next_runs_given','next_wkts','Selection' ,'Credits']]

# Replace 'next_runs_given' with 0 if 'next_overs' is 0.0
test2['next_runs_given'] = np.where(test2['next_overs'] == 0.0, 0.0, test2['next_runs_given'])

# Create additional features''''

test2.insert(loc=6, column='30 runs', value=np.where(test2['next_runs'] > -30, 1, 0))


test2.insert(loc=7, column='50 runs', value=np.where(test2['next_runs'] >= 50, 1, 0))
test2.insert(loc=8, column='100 runs', value=np.where(test2['next_runs'] >= 100, 1, 0))
test2.insert(loc=9, column='Duck', value=np.where(test2['next_runs'] == 0, 1, 0))
test2.insert(loc=13, column='3 wickets', value=np.where(test2['next_wkts'] >= 3, 1, 0))
test2.insert(loc=14, column='4 wickets', value=np.where(test2['next_wkts'] >= 4, 1, 0))
test2.insert(loc=15, column='5 wickets', value=np.where(test2['next_wkts'] >= 5, 1, 0))
test2.insert(loc=10, column='Bat SR', value=round(test2['next_runs'] / test2['next_balls'] * 100, 1))
test2.insert(loc=14, column='Econ', value=round(test2['next_runs_given'] / test2['next_overs'], 1))
test2.insert(loc=6, column='4s Scored', value=fours_model.predict(test2[test2.columns[4:6]]))
test2.insert(loc=7, column='6s Scored', value=fours_model.predict(test2[test2.columns[4:6]]))
test2.insert(loc=12, column='Out', value=out_model.predict(test2[test2.columns[1:12]]))



test2 = test2.fillna(0)
test2 = test2.replace([np.inf, -np.inf], 0)
df_model.fillna(0, inplace=True)
test2['Predicted_Points'] = points_model.predict(test2[test2.columns[1:]])
test2 = pd.merge(test2, totali[['Player', 'Team']], on='Player', how='left')
test2 = pd.get_dummies(test2, columns=['Team'], drop_first=True, prefix=['Team'])
test2['4s Scored'] = round(test2['4s Scored'], 0)
test2['6s Scored'] = round(test2['6s Scored'], 0)

# Display the DataFrame


test2

C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1113247824.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['next_runs_given'] = np.where(test2['next_overs'] == 0.0, 0.0, test2['next_runs_given'])
C:\Users\anilk\AppData\Local\Temp\ipykernel_16564\1113247824.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.fillna(0, inplace=True)


,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Predicted_Points,Team_SA
0,S Mandhana,True,False,False,136.0,120.0,14.0,14.0,1,0,...,0,46.0,42.0,0.0,0.0,0.0,72.0,8.5,202.892274,False
1,Shafali Verma,True,False,False,20.0,38.0,1.0,1.0,1,1,...,0,68.0,128.0,1.0,4.0,0.0,68.0,8.0,35.777791,False
2,D Hemalatha,True,False,False,24.0,41.0,2.0,2.0,1,0,...,0,24.0,41.0,0.0,0.0,0.0,1.0,7.5,31.047960,False
3,H Kaur,True,False,False,103.0,88.0,11.0,11.0,1,0,...,0,-2.0,-1.0,1.0,12.0,0.0,85.0,7.5,140.902441,False
4,Richa Ghosh,False,False,True,25.0,13.0,3.0,3.0,1,0,...,0,10.0,12.0,0.0,0.0,0.0,44.0,7.5,38.018504,False
5,JI Rodrigues,True,False,False,17.0,28.0,1.0,1.0,1,0,...,0,18.0,18.0,1.0,1.0,1.0,59.0,7.0,24.516600,False
6,Deepti Sharma,False,False,False,0.0,0.0,0.0,0.0,1,0,...,0,3.0,-0.0,5.0,12.0,0.0,96.0,8.0,54.986168,False
7,P Vastrakar,False,False,False,0.0,0.0,0.0,0.0,1,0,...,0,2.0,1.0,2.0,10.0,0.0,16.0,7.5,56.765613,False
8,A Reddy,False,True,False,31.0,42.0,3.0,3.0,1,0,...,0,3.0,2.0,0.0,0.0,0.0,42.0,8.0,43.190834,False
9,RP Yadav,False,True,False,0.0,0.0,0.0,0.0,1,0,...,0,0.0,0.0,10.0,52.0,0.0,4.0,6.0,5.445357,False


In [85]:
test2[['Player', 'Team_SA' ,'Credits', 'Predicted_Points']].sort_values('Predicted_Points', ascending=False)


,Player,Team_SA,Credits,Predicted_Points
0,S Mandhana,False,8.5,202.892274
12,L Wolvaardt,True,8.5,198.031103
16,M Kapp,True,7.0,150.598761
3,H Kaur,False,7.5,140.902441
20,N Mlaba,True,8.0,61.387179
7,P Vastrakar,False,7.5,56.765613
6,Deepti Sharma,False,8.0,54.986168
17,N de Klerk,True,8.5,43.755426
8,A Reddy,False,8.0,43.190834
4,Richa Ghosh,False,7.5,38.018504


In [86]:
playing11 =['L Wolvaardt', 'Tazmin Brits', 'Anneke Bosch', 'Sune Luus', 'M Kapp', 'N de Klerk', 'N Shangase', 'M Ridder', 'T Sekhukhune', 'N Mlaba', 'A Khaka', 'S Mandhana', 'Shafali Verma', 'P Punia', 'H Kaur', 'Richa Ghosh', 'JI Rodrigues', 'Deepti Sharma', 'P Vastrakar', 'RP Yadav', 'Shreyanka Patil', 'Arundhati Reddy']
test2 = test2[test2['Player'].isin(playing11)]
# Check the respective points of all players
test2


,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Predicted_Points,Team_SA
0,S Mandhana,True,False,False,136.0,120.0,14.0,14.0,1,0,...,0,46.0,42.0,0.0,0.0,0.0,72.0,8.5,202.892274,False
1,Shafali Verma,True,False,False,20.0,38.0,1.0,1.0,1,1,...,0,68.0,128.0,1.0,4.0,0.0,68.0,8.0,35.777791,False
3,H Kaur,True,False,False,103.0,88.0,11.0,11.0,1,0,...,0,-2.0,-1.0,1.0,12.0,0.0,85.0,7.5,140.902441,False
4,Richa Ghosh,False,False,True,25.0,13.0,3.0,3.0,1,0,...,0,10.0,12.0,0.0,0.0,0.0,44.0,7.5,38.018504,False
5,JI Rodrigues,True,False,False,17.0,28.0,1.0,1.0,1,0,...,0,18.0,18.0,1.0,1.0,1.0,59.0,7.0,24.516600,False
6,Deepti Sharma,False,False,False,0.0,0.0,0.0,0.0,1,0,...,0,3.0,-0.0,5.0,12.0,0.0,96.0,8.0,54.986168,False
7,P Vastrakar,False,False,False,0.0,0.0,0.0,0.0,1,0,...,0,2.0,1.0,2.0,10.0,0.0,16.0,7.5,56.765613,False
9,RP Yadav,False,True,False,0.0,0.0,0.0,0.0,1,0,...,0,0.0,0.0,10.0,52.0,0.0,4.0,6.0,5.445357,False
12,L Wolvaardt,True,False,False,135.0,135.0,14.0,14.0,1,1,...,0,124.0,107.0,0.0,0.0,0.0,8.0,8.5,198.031103,True
13,Tazmin Brits,True,False,False,5.0,11.0,0.0,0.0,1,0,...,0,30.0,43.0,0.0,0.0,0.0,70.0,8.5,10.030313,True


In [87]:
test2[['Player', 'next_runs','Team_SA', 'Credits', 'Predicted_Points']].sort_values('Predicted_Points', ascending=False)

,Player,next_runs,Team_SA,Credits,Predicted_Points
0,S Mandhana,46.0,False,8.5,202.892274
12,L Wolvaardt,124.0,True,8.5,198.031103
16,M Kapp,-5.0,True,7.0,150.598761
3,H Kaur,-2.0,False,7.5,140.902441
20,N Mlaba,2.0,True,8.0,61.387179
7,P Vastrakar,2.0,False,7.5,56.765613
6,Deepti Sharma,3.0,False,8.0,54.986168
17,N de Klerk,13.0,True,8.5,43.755426
4,Richa Ghosh,10.0,False,7.5,38.018504
1,Shafali Verma,68.0,False,8.0,35.777791


In [88]:
import pulp as p
test3=pd.DataFrame()
test3=test2
Lp_prob = p.LpProblem('Problem', p.LpMaximize)
test3=test3.reset_index()

In [89]:
import pulp as p
import pandas as pd

# Initialize the linear programming problem
Lp_prob = p.LpProblem('Dream11_Team_Selection', p.LpMaximize)

# Decision Variables
decision_variables = {row['Player']: p.LpVariable(row['Player'], cat='Binary') for index, row in test3.iterrows()}

# Objective Function: Maximize predicted points
total_points = p.lpSum(row['Predicted_Points'] * decision_variables[row['Player']] for index, row in test3.iterrows())
Lp_prob += total_points

# Constraints
# Minimum 3 and maximum 6 batsmen
batsmen_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_BAT'] == 1)
Lp_prob += batsmen_constraint >= 3
Lp_prob += batsmen_constraint <= 6

# Minimum 3 and maximum 6 bowlers
bowlers_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_BOWL'] == 1)
Lp_prob += bowlers_constraint >= 3
Lp_prob += bowlers_constraint <= 6

# Minimum 1 and maximum 4 wicketkeepers
wicketkeepers_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_WK'] == 1)
Lp_prob += wicketkeepers_constraint >= 1
Lp_prob += wicketkeepers_constraint <= 4

# 11 players
Lp_prob += p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows()) == 11

# Sum of credits to be within 100
credits_constraint = p.lpSum(row['Credits'] * decision_variables[row['Player']] for index, row in test3.iterrows())
Lp_prob += credits_constraint <= 100

# Minimum 4 and maximum 7 players from one team
team_constraints = {}
for team in set(test3['Team_SA']):
    team_constraints[team] = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Team_SA'] == team)
    Lp_prob += team_constraints[team] >= 4
    Lp_prob += team_constraints[team] <= 7

# Solve the optimization problem
optimization_result = Lp_prob.solve()

# Create a DataFrame for selected players
selected_players_df = pd.concat([test3.loc[test3['Player'] == player, ['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Selection', 'Credits', 'Predicted_Points', 'Team_SA']] for player in decision_variables if p.value(decision_variables[player]) == 1], ignore_index=True)

# Assign values based on the predicted points
selected_players_df['value'] = selected_players_df['Predicted_Points'].apply(lambda x: 2.0 if x == selected_players_df['Predicted_Points'].max() else (1.5 if x == selected_players_df['Predicted_Points'].nlargest(2).iloc[-1] else 1.0))

# Print the selected players DataFrame
print("Selected Players:")
selected_players_df

Selected Players:


,Player,Position_BAT,Position_BOWL,Position_WK,Selection,Credits,Predicted_Points,Team_SA,value
0,S Mandhana,True,False,False,72.0,8.5,202.892274,False,2.0
1,H Kaur,True,False,False,85.0,7.5,140.902441,False,1.0
2,Richa Ghosh,False,False,True,44.0,7.5,38.018504,False,1.0
3,Deepti Sharma,False,False,False,96.0,8.0,54.986168,False,1.0
4,P Vastrakar,False,False,False,16.0,7.5,56.765613,False,1.0
5,RP Yadav,False,True,False,4.0,6.0,5.445357,False,1.0
6,L Wolvaardt,True,False,False,8.0,8.5,198.031103,True,1.5
7,M Kapp,False,False,False,94.0,7.0,150.598761,True,1.0
8,N de Klerk,False,False,False,94.0,8.5,43.755426,True,1.0
9,N Mlaba,False,True,False,94.0,8.0,61.387179,True,1.0


In [90]:
selected_players_df[['Player', 'Position_BAT','Position_BOWL','Position_WK'	,'Selection','Credits','Predicted_Points','Team_SA','value']].sort_values('Predicted_Points', ascending=False)

,Player,Position_BAT,Position_BOWL,Position_WK,Selection,Credits,Predicted_Points,Team_SA,value
0,S Mandhana,True,False,False,72.0,8.5,202.892274,False,2.0
6,L Wolvaardt,True,False,False,8.0,8.5,198.031103,True,1.5
7,M Kapp,False,False,False,94.0,7.0,150.598761,True,1.0
1,H Kaur,True,False,False,85.0,7.5,140.902441,False,1.0
9,N Mlaba,False,True,False,94.0,8.0,61.387179,True,1.0
4,P Vastrakar,False,False,False,16.0,7.5,56.765613,False,1.0
3,Deepti Sharma,False,False,False,96.0,8.0,54.986168,False,1.0
8,N de Klerk,False,False,False,94.0,8.5,43.755426,True,1.0
2,Richa Ghosh,False,False,True,44.0,7.5,38.018504,False,1.0
10,A Khaka,False,True,False,94.0,8.0,7.234302,True,1.0
